## Setup

### Imports

In [1]:
import os
import json
import glob
import torch
import re
import einops
import pandas as pd
from functools import partial
from torch import Tensor
from torchtyping import TensorType as TT


import plotly.express as px

from utils.data_utils import generate_data_and_caches
from utils.data_processing import (
    load_edge_scores_into_dictionary,
)
from utils.visualization import plot_attention_heads, imshow_p
from utils.backup_analysis import (
    load_model,
    run_iteration,
    process_backup_results,
    get_past_nmhs_for_checkpoints,
    plot_top_heads
)

### Parameters

In [2]:
TASK = 'ioi'
PERFORMANCE_METRIC = 'logit_diff'
BASE_MODEL = "pythia-160m"
VARIANT = None
CACHE = "model_cache"
IOI_DATASET_SIZE = 70

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

torch.set_grad_enabled(False)

### Functions

## Experiments

### Experiment Parameters

In [3]:
TASK = 'ioi'
PERFORMANCE_METRIC = 'logit_diff'
BASE_MODEL = "pythia-160m"
VARIANT = "EleutherAI/pythia-160m-alldropout"
MODEL_SHORTNAME = BASE_MODEL if not VARIANT else VARIANT[11:]
CACHE = "model_cache"
IOI_DATASET_SIZE = 70
COPY_SCORE_THRESHOLD = 75.0

### Circuit Data

In [55]:
folder_path = f'results/graphs/{MODEL_SHORTNAME}/{TASK}'
df = load_edge_scores_into_dictionary(folder_path)

# filter everything before 1000 steps
df = df[df['checkpoint'] >= 1000]

df[['source', 'target']] = df['edge'].str.split('->', expand=True)
len(df['target'].unique())

Processing file 1/143: results/graphs/pythia-160m-alldropout/ioi/57000.json
Processing file 2/143: results/graphs/pythia-160m-alldropout/ioi/141000.json
Processing file 3/143: results/graphs/pythia-160m-alldropout/ioi/95000.json
Processing file 4/143: results/graphs/pythia-160m-alldropout/ioi/107000.json
Processing file 5/143: results/graphs/pythia-160m-alldropout/ioi/34000.json
Processing file 6/143: results/graphs/pythia-160m-alldropout/ioi/6000.json
Processing file 7/143: results/graphs/pythia-160m-alldropout/ioi/37000.json
Processing file 8/143: results/graphs/pythia-160m-alldropout/ioi/39000.json
Processing file 9/143: results/graphs/pythia-160m-alldropout/ioi/104000.json
Processing file 10/143: results/graphs/pythia-160m-alldropout/ioi/59000.json
Processing file 11/143: results/graphs/pythia-160m-alldropout/ioi/67000.json
Processing file 12/143: results/graphs/pythia-160m-alldropout/ioi/111000.json
Processing file 13/143: results/graphs/pythia-160m-alldropout/ioi/76000.json
Proce

445

### Dataset Setup

In [56]:
initial_model = load_model(BASE_MODEL, VARIANT, 143000, CACHE, device)
size=70
ioi_dataset, abc_dataset = generate_data_and_caches(initial_model, size, verbose=True)

Loaded model EleutherAI/pythia-160m-alldropout at step143000; now loading into HookedTransformer


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-160m into HookedTransformer


In [57]:
# imshow_p(
#     per_head_ablated_logit_diffs,
#     title="Headwise logit diff contribution, post NMH KO",
#     labels={"x": "Head", "y": "Layer", "color": "Logit diff attribution"},
#     #coloraxis=dict(colorbar_ticksuffix = "%"),
#     border=True,
#     width=600,
#     margin={"r": 100, "l": 100}
# )

### Run Experiment

In [58]:
experiment_metrics = dict()
# create folder
os.makedirs(f'results/backup/{MODEL_SHORTNAME}', exist_ok=True)

for checkpoint in range(4000, 144000, 1000):

    experiment_metrics = run_iteration(
        BASE_MODEL, VARIANT, df, checkpoint=checkpoint, dataset=ioi_dataset, experiment_metrics=experiment_metrics, 
        threshold=COPY_SCORE_THRESHOLD
    )
    experiment_metrics = process_backup_results(df, checkpoint, experiment_metrics)

    # save to file, using pytorch format
    torch.save(experiment_metrics, f'results/backup/{MODEL_SHORTNAME}/nmh_backup_metrics.pt')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded model EleutherAI/pythia-160m-alldropout at step4000; now loading into HookedTransformer
Loaded pretrained model pythia-160m into HookedTransformer
Copy circuit for head 9.8 (sign=1) : Top 5 accuracy: 47.61904761904761%
Copy circuit for head 9.11 (sign=1) : Top 5 accuracy: 88.57142857142857%
Copy circuit for head 9.6 (sign=1) : Top 5 accuracy: 2.380952380952381%
Copy circuit for head 9.4 (sign=1) : Top 5 accuracy: 82.85714285714286%
Copy circuit for head 7.6 (sign=1) : Top 5 accuracy: 92.85714285714286%
Copy circuit for head 7.10 (sign=1) : Top 5 accuracy: 100.0%
Copy circuit for head 8.9 (sign=1) : Top 5 accuracy: 100.0%
Checkpoint 4000:
Heads ablated:            [(9, 11), (9, 4), (7, 6), (7, 10), (8, 9)]
Original logit diff:      0.7503997087
Post ablation logit diff: 0.8393040299
Logit diff % change:      11.85%


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded model EleutherAI/pythia-160m-alldropout at step5000; now loading into HookedTransformer
Loaded pretrained model pythia-160m into HookedTransformer
Checkpoint 5000:
Heads ablated:            []
Original logit diff:      1.5525176525
Post ablation logit diff: 1.5525176525
Logit diff % change:      0.00%


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Loaded model EleutherAI/pythia-160m-alldropout at step141000; now loading into HookedTransformer
Loaded pretrained model pythia-160m into HookedTransformer
Copy circuit for head 9.4 (sign=1) : Top 5 accuracy: 90.0%
Copy circuit for head 8.9 (sign=1) : Top 5 accuracy: 100.0%
Copy circuit for head 8.8 (sign=1) : Top 5 accuracy: 50.0%
Copy circuit for head 8.5 (sign=1) : Top 5 accuracy: 86.19047619047619%
Copy circuit for head 8.1 (sign=1) : Top 5 accuracy: 98.57142857142858%
Copy circuit for head 7.10 (sign=1) : Top 5 accuracy: 100.0%
Copy circuit for head 7.9 (sign=1) : Top 5 accuracy: 73.33333333333333%
Copy circuit for head 7.6 (sign=1) : Top 5 accuracy: 2.380952380952381%
Copy circuit for head 9.6 (sign=1) : Top 5 accuracy: 83.33333333333334%
Copy circuit for head 9.9 (sign=1) : Top 5 accuracy: 87.61904761904762%
Copy circuit for head 9.11 (sign=1) : Top 5 accuracy: 94.76190476190476%
Copy circuit for head 10.10 (sign=1) : Top 5 accuracy: 0.4761904761904762%
Checkpoint 141000:
Heads 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded model EleutherAI/pythia-160m-alldropout at step142000; now loading into HookedTransformer
Loaded pretrained model pythia-160m into HookedTransformer
Copy circuit for head 9.11 (sign=1) : Top 5 accuracy: 94.76190476190476%
Copy circuit for head 9.9 (sign=1) : Top 5 accuracy: 86.19047619047619%
Copy circuit for head 7.6 (sign=1) : Top 5 accuracy: 3.3333333333333335%
Copy circuit for head 7.9 (sign=1) : Top 5 accuracy: 75.23809523809524%
Copy circuit for head 7.10 (sign=1) : Top 5 accuracy: 100.0%
Copy circuit for head 8.1 (sign=1) : Top 5 accuracy: 98.57142857142858%
Copy circuit for head 8.5 (sign=1) : Top 5 accuracy: 85.23809523809524%
Copy circuit for head 8.8 (sign=1) : Top 5 accuracy: 52.38095238095239%
Copy circuit for head 8.9 (sign=1) : Top 5 accuracy: 100.0%
Copy circuit for head 9.4 (sign=1) : Top 5 accuracy: 89.52380952380953%
Copy circuit for head 9.6 (sign=1) : Top 5 accuracy: 82.38095238095238%
Checkpoint 142000:
Heads ablated:            [(9, 11), (9, 9), (7, 9), (7

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded model EleutherAI/pythia-160m-alldropout at step143000; now loading into HookedTransformer
Loaded pretrained model pythia-160m into HookedTransformer
Copy circuit for head 5.11 (sign=1) : Top 5 accuracy: 59.523809523809526%
Copy circuit for head 9.11 (sign=1) : Top 5 accuracy: 94.76190476190476%
Copy circuit for head 9.9 (sign=1) : Top 5 accuracy: 86.19047619047619%
Copy circuit for head 7.6 (sign=1) : Top 5 accuracy: 2.380952380952381%
Copy circuit for head 7.9 (sign=1) : Top 5 accuracy: 75.23809523809524%
Copy circuit for head 7.10 (sign=1) : Top 5 accuracy: 100.0%
Copy circuit for head 8.1 (sign=1) : Top 5 accuracy: 98.57142857142858%
Copy circuit for head 8.2 (sign=1) : Top 5 accuracy: 99.52380952380952%
Copy circuit for head 8.5 (sign=1) : Top 5 accuracy: 86.19047619047619%
Copy circuit for head 8.8 (sign=1) : Top 5 accuracy: 52.85714285714286%
Copy circuit for head 8.9 (sign=1) : Top 5 accuracy: 100.0%
Copy circuit for head 9.4 (sign=1) : Top 5 accuracy: 90.95238095238095%


## View Results

#### Pythia 160m

In [14]:
MODEL_TO_VIEW = "pythia-160m-alldropout"

In [15]:
experiment_metrics = torch.load(f'results/backup/{MODEL_TO_VIEW}/nmh_backup_metrics.pt')

In [16]:
experiment_metrics[4000].keys()

dict_keys(['logit_diff', 'per_head_logit_diffs', 'ablation_targets', 'ablated_logit_diff', 'per_head_ablated_logit_diffs', 'per_head_logit_diff_delta', 'in_circuit_head_delta', 'outside_circuit_head_delta', 'summed_in_circuit_head_delta', 'summed_outside_circuit_head_delta', 'summed_total_head_delta'])

In [17]:
summed_in_circuit_head_deltas = {checkpoint: experiment_metrics[checkpoint]["summed_in_circuit_head_delta"] for checkpoint in experiment_metrics.keys()}
summed_outside_circuit_head_deltas = {checkpoint: experiment_metrics[checkpoint]["summed_outside_circuit_head_delta"] for checkpoint in experiment_metrics.keys()}
summed_total_head_deltas = {checkpoint: experiment_metrics[checkpoint]["summed_total_head_delta"] for checkpoint in experiment_metrics.keys()}
per_head_logit_diff_deltas = {checkpoint: experiment_metrics[checkpoint]["per_head_logit_diff_delta"] for checkpoint in experiment_metrics.keys()}
total_logit_diff_deltas = {checkpoint: experiment_metrics[checkpoint]['ablated_logit_diff'] - experiment_metrics[checkpoint]['logit_diff'] for checkpoint in experiment_metrics.keys()}

for checkpoint in experiment_metrics.keys():
    # divide by total original logit diff
    summed_in_circuit_head_deltas[checkpoint] = summed_in_circuit_head_deltas[checkpoint] / experiment_metrics[checkpoint]["logit_diff"]
    summed_outside_circuit_head_deltas[checkpoint] = summed_outside_circuit_head_deltas[checkpoint] / experiment_metrics[checkpoint]["logit_diff"]
    summed_total_head_deltas[checkpoint] = summed_total_head_deltas[checkpoint] / experiment_metrics[checkpoint]["logit_diff"]
    per_head_logit_diff_deltas[checkpoint] = per_head_logit_diff_deltas[checkpoint] / experiment_metrics[checkpoint]["logit_diff"]
    total_logit_diff_deltas[checkpoint] = total_logit_diff_deltas[checkpoint] / experiment_metrics[checkpoint]["logit_diff"]

In [18]:
# plot summed_in_circuit_head_deltas with plotly express
fig = px.line(
    x=list(summed_in_circuit_head_deltas.keys()), 
    y=list(summed_in_circuit_head_deltas.values()), 
    title="Summed Post-NMH-Ablation In-Circuit Head Logit Diff Change Over Time (Pythia 160m Default)",
    labels={'x': 'Checkpoint', 'y': 'Change as % of original logit diff'} 
)
fig.show()


In [19]:
# plot summed_outside_circuit_head_deltas
fig = px.line(
    x=list(summed_outside_circuit_head_deltas.keys()), 
    y=list(summed_outside_circuit_head_deltas.values()), 
    title="Summed Post-NMH-Ablation Outside-Circuit Head Attribution Change (Pythia 160m Default)",
    labels={'x': 'Checkpoint', 'y': 'Change as % of original logit diff'} 
)
fig.show()

In [20]:
# plot total_head_deltas
fig = px.line(
    x=list(summed_total_head_deltas.keys()), 
    y=list(summed_total_head_deltas.values()), 
    title="Summed Total Post-NMH-Ablation Head Attribution Change (Pythia 160m Default)",
    labels={'x': 'Checkpoint', 'y': 'Change as % of original logit diff'}
)

fig.show()

In [21]:
cumulative_nmhs, checkpoint_nmhs = get_past_nmhs_for_checkpoints(experiment_metrics)

{(8, 1), (7, 10), (9, 4), (8, 9), (9, 8), (9, 11), (8, 2)}


In [12]:
top_backup_heads = plot_top_heads(MODEL_TO_VIEW, checkpoint_dict=per_head_logit_diff_deltas, cumulative_nmhs=cumulative_nmhs)

In [68]:
top_backup_heads[top_backup_heads['Previous NMH']==True].head(50)

,Checkpoint,Layer-Head,Layer,Head,Value,Previous NMH,Checkpoint_sum,Value_sum,Previous NMH_sum,Top K
36,11000,Layer 9-Head 9,9,9,0.007299,True,5602000,0.951745,78,True
39,12000,Layer 9-Head 4,9,4,0.020281,True,2013000,0.203263,22,False
44,13000,Layer 9-Head 9,9,9,0.008192,True,5602000,0.951745,78,True
46,13000,Layer 9-Head 4,9,4,0.010593,True,2013000,0.203263,22,False
50,14000,Layer 9-Head 9,9,9,0.012010,True,5602000,0.951745,78,True
57,15000,Layer 9-Head 9,9,9,0.012112,True,5602000,0.951745,78,True
67,17000,Layer 9-Head 9,9,9,0.013145,True,5602000,0.951745,78,True
71,18000,Layer 9-Head 4,9,4,0.013086,True,2013000,0.203263,22,False
72,18000,Layer 9-Head 9,9,9,0.012296,True,5602000,0.951745,78,True
77,19000,Layer 9-Head 9,9,9,0.012480,True,5602000,0.951745,78,True


In [13]:
checkpoint_nmhs

{4000: {(7, 6), (7, 10), (8, 9), (9, 4), (9, 11)},
 5000: set(),
 6000: {(7, 6), (8, 9), (9, 4), (9, 11)},
 7000: {(7, 10), (8, 9), (9, 4), (9, 11)},
 8000: {(8, 9), (9, 4), (9, 11)},
 9000: {(8, 9), (9, 4), (9, 9), (9, 11)},
 10000: {(8, 9), (9, 4), (9, 8), (9, 9), (9, 11)},
 11000: {(8, 9), (9, 4), (9, 8), (9, 11)},
 12000: {(8, 1), (8, 9), (9, 8), (9, 9), (9, 11)},
 13000: {(8, 9), (9, 8), (9, 11)},
 14000: {(8, 9), (9, 4), (9, 8), (9, 11)},
 15000: {(8, 2), (8, 9), (9, 4), (9, 8), (9, 11)},
 16000: {(8, 9), (9, 4), (9, 8), (9, 9), (9, 11)},
 17000: {(8, 2), (8, 9), (9, 4), (9, 8), (9, 11)},
 18000: {(8, 2), (8, 9), (9, 8), (9, 11)},
 19000: {(8, 2), (8, 9), (9, 4), (9, 8), (9, 11)},
 20000: {(7, 10), (8, 1), (8, 2), (8, 9), (9, 4), (9, 8), (9, 11)},
 21000: {(8, 2), (8, 9), (9, 4), (9, 8), (9, 9), (9, 11)},
 22000: {(8, 1), (8, 2), (8, 9), (9, 4), (9, 8), (9, 11)},
 23000: {(8, 1), (8, 2), (8, 9), (9, 8), (9, 11)},
 24000: {(7, 10), (8, 1), (8, 2), (8, 9), (9, 4), (9, 8), (9, 11)},